In [21]:
import re
import tensorflow as tf
import tensorflow_hub as hub
import os
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [22]:
# Constants
group_code = "santiagomartinez_201533279_camilocastaneda_202314092"
segment_length = 200

In [23]:
# Function to preprocess text
def processing_text(texto):
    # Paso 1: Remover con un expresión regular carateres especiales (no palabras).
    processed_feature = re.sub(r'\W', ' ', str(texto))
    # Paso 2: Remover ocurrencias de caracteres individuales
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)
    # Paso 3: Remover números (Ocurrencias muy esporádicas en nuestro dataset)
    processed_feature = re.sub(r'[0-9]+', ' ', processed_feature)
    # Paso 4: Simplificar espacios concecutivos a un único espacio entre palabras
    processed_feature = re.sub(' +', ' ', processed_feature)
    # Paso 5: Pasar todo el texto a minúsculas
    processed_feature = processed_feature.lower()

    return processed_feature

In [24]:
# Load and preprocess the text data
def load_and_preprocess_text_data(folder_path, segment_length):
    texts = []
    labels = []

    for author in authors:
        author_folder = os.path.join(folder_path, author)
        for filename in os.listdir(author_folder):
            with open(os.path.join(author_folder, filename), "r", encoding="utf-8") as file:
                text = file.read()
                # Process the text
                processed_text = processing_text(text)
                # Split text into segments if needed
                sequences = [processed_text[i:i+segment_length] for i in range(0, len(processed_text), segment_length)]
                texts.extend(sequences)
                labels.extend([author] * len(sequences))

    return texts, labels

# Read and preprocess the text data
def get_corpus(folder_path):
    texts = []
    invalid_txt = ['']
    
    for root, _, files in os.walk(folder_path):
        for file_ in files:
            with open(os.path.join(root, file_), "r", encoding="utf-8") as f:
                for line in f:
                    pre_processed_text = processing_text(line)
                    if pre_processed_text not in invalid_txt:
                        texts.append(pre_processed_text)
    
    return texts

In [25]:
# Get authors
authors = os.listdir("book_datasets")

# Initialize and fit the Keras tokenizer
tokenizer = Tokenizer()
corpus = get_corpus("book_datasets")
corpus = [sentence for sentence in corpus if sentence!=' ' and sentence!='' and len(sentence.split())>3]
tokenizer.fit_on_texts(corpus)
vocab_size = len(tokenizer.word_index) + 1
print("vocab size: " + str(vocab_size))
# Load and preprocess the text data
texts, labels = load_and_preprocess_text_data("book_datasets", segment_length)

vocab size: 20628


In [26]:
# Encode labels and one-hot encode
label_encoder = LabelEncoder()
label_encoder.fit(authors)
labels_encoded = label_encoder.transform(labels)
labels_one_hot = to_categorical(labels_encoded, num_classes=len(authors))


In [27]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels_one_hot, test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(test_texts, test_labels, test_size=0.5, random_state=42)

In [28]:
# Convert the text data into numpy arrays
train_texts = np.array(train_texts)
train_labels = np.array(train_labels)
val_texts = np.array(val_texts)
val_labels = np.array(val_labels)
test_texts = np.array(test_texts)
test_labels = np.array(test_labels)

In [30]:
# Load the pre-trained Word2Vec model from TensorFlow Hub
embedding_model_url = "https://tfhub.dev/google/Wiki-words-500/2"
hub_layer = hub.KerasLayer(embedding_model_url, input_shape=[], dtype=tf.string, trainable=False)

# Define your feed forward neural network
model = tf.keras.Sequential([
    hub_layer,  # The Word2Vec embedding layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(authors), activation='softmax')
])

model.summary()
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 500)               504687500 
                                                                 
 dense_3 (Dense)             (None, 128)               64128     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dropout_4 (Dropout)         (None, 64)               

In [31]:
# Train the model
history = model.fit(
    train_texts, train_labels,
    epochs=100,  # You can adjust the number of epochs
    batch_size=64,
    validation_data=(val_texts, val_labels)
)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_texts, test_labels)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/100
184/184 [==============================] - 2s 6ms/step - loss: 0.9371 - accuracy: 0.5474 - val_loss: 0.8291 - val_accuracy: 0.6372
Epoch 2/100
184/184 [==============================] - 1s 5ms/step - loss: 0.8094 - accuracy: 0.6367 - val_loss: 0.7190 - val_accuracy: 0.6712
Epoch 3/100
184/184 [==============================] - 1s 5ms/step - loss: 0.7630 - accuracy: 0.6504 - val_loss: 0.6746 - val_accuracy: 0.6855
Epoch 4/100
184/184 [==============================] - 1s 5ms/step - loss: 0.7322 - accuracy: 0.6653 - val_loss: 0.6444 - val_accuracy: 0.7147
Epoch 5/100
184/184 [==============================] - 1s 5ms/step - loss: 0.6977 - accuracy: 0.6917 - val_loss: 0.6157 - val_accuracy: 0.7249
Epoch 6/100
184/184 [==============================] - 1s 5ms/step - loss: 0.6601 - accuracy: 0.7150 - val_loss: 0.5656 - val_accuracy: 0.7717
Epoch 7/100
184/184 [==============================] - 1s 5ms/step - loss: 0.6329 - accuracy: 0.7231 - val_loss: 0.5091 - val_accuracy: 0.7901

In [36]:
# Load the pre-trained Word2Vec model from TensorFlow Hub
embedding_model_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding_model_url, input_shape=[], dtype=tf.string, trainable=True)

# Define your feed forward neural network
model = tf.keras.Sequential([
    hub_layer,  # The Word2Vec embedding layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(authors), activation='softmax')
])

model.summary()
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_15 (Dense)            (None, 128)               2688      
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dropout_12 (Dropout)        (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 3)                 195       
                                                                 
Total params: 411159 (1.57 MB)
Trainable params: 41115

In [37]:
# Train the model
history = model.fit(
    train_texts, train_labels,
    epochs=100,  # You can adjust the number of epochs
    batch_size=64,
    validation_data=(val_texts, val_labels)
)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_texts, test_labels)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/100


184/184 [==============================] - 2s 7ms/step - loss: 1.0057 - accuracy: 0.5134 - val_loss: 0.8696 - val_accuracy: 0.5965
Epoch 2/100
184/184 [==============================] - 1s 6ms/step - loss: 0.8603 - accuracy: 0.6063 - val_loss: 0.7125 - val_accuracy: 0.7058
Epoch 3/100
184/184 [==============================] - 1s 6ms/step - loss: 0.7074 - accuracy: 0.6972 - val_loss: 0.5547 - val_accuracy: 0.7724
Epoch 4/100
184/184 [==============================] - 1s 6ms/step - loss: 0.5692 - accuracy: 0.7598 - val_loss: 0.4653 - val_accuracy: 0.8010
Epoch 5/100
184/184 [==============================] - 1s 6ms/step - loss: 0.4698 - accuracy: 0.8076 - val_loss: 0.4227 - val_accuracy: 0.8071
Epoch 6/100
184/184 [==============================] - 1s 6ms/step - loss: 0.3964 - accuracy: 0.8404 - val_loss: 0.3994 - val_accuracy: 0.8315
Epoch 7/100
184/184 [==============================] - 1s 6ms/step - loss: 0.3356 - accuracy: 0.8640 - val_loss: 0.3765 - val_accuracy: 0.8329
Epoch 8/100

In [38]:
# Load the pre-trained Word2Vec model from TensorFlow Hub
embedding_model_url = "https://tfhub.dev/google/Wiki-words-250/2"
hub_layer = hub.KerasLayer(embedding_model_url, input_shape=[], dtype=tf.string, trainable=False)

# Define your feed forward neural network
model = tf.keras.Sequential([
    hub_layer,  # The Word2Vec embedding layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(authors), activation='softmax')
])

model.summary()
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_4 (KerasLayer)  (None, 250)               252343750 
                                                                 
 dense_18 (Dense)            (None, 128)               32128     
                                                                 
 dropout_13 (Dropout)        (None, 128)               0         
                                                                 
 dense_19 (Dense)            (None, 64)                8256      
                                                                 
 dropout_14 (Dropout)        (None, 64)                0         
                                                                 
 dense_20 (Dense)            (None, 3)                 195       
                                                                 
Total params: 252384329 (962.77 MB)
Trainable params: 

In [39]:
# Train the model
history = model.fit(
    train_texts, train_labels,
    epochs=100,  # You can adjust the number of epochs
    batch_size=64,
    validation_data=(val_texts, val_labels)
)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_texts, test_labels)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/100
184/184 [==============================] - 1s 5ms/step - loss: 0.9060 - accuracy: 0.5748 - val_loss: 0.7556 - val_accuracy: 0.6508
Epoch 2/100
184/184 [==============================] - 1s 4ms/step - loss: 0.7873 - accuracy: 0.6430 - val_loss: 0.6768 - val_accuracy: 0.6882
Epoch 3/100
184/184 [==============================] - 1s 4ms/step - loss: 0.7286 - accuracy: 0.6776 - val_loss: 0.6230 - val_accuracy: 0.7296
Epoch 4/100
184/184 [==============================] - 1s 4ms/step - loss: 0.6890 - accuracy: 0.6966 - val_loss: 0.5952 - val_accuracy: 0.7432
Epoch 5/100
184/184 [==============================] - 1s 4ms/step - loss: 0.6568 - accuracy: 0.7135 - val_loss: 0.5614 - val_accuracy: 0.7643
Epoch 6/100
184/184 [==============================] - 1s 4ms/step - loss: 0.6403 - accuracy: 0.7216 - val_loss: 0.5689 - val_accuracy: 0.7683
Epoch 7/100
184/184 [==============================] - 1s 4ms/step - loss: 0.6159 - accuracy: 0.7323 - val_loss: 0.5279 - val_accuracy: 0.7880

Podemos observar que el desempeño de distintos modelos preentrenados "word2vec", es menor al desempeño usando embeddings entrenedaos directamente en el texto objetivo. esto puede ser debido a que estos embeddigns están diseñados para generalizar en muchos otros contextos, lo que remueve parcialmente poder de representación que puede explotar el modelo entrenado directamente sobre el corpus a analizar.